In [8]:
import numpy as np
import random
qubit_0 =  np.array([1,0]).reshape(2,)
qubit_1 =  np.array([0,1]).reshape(2,)
qubit_000 = np.kron(np.kron(qubit_0,qubit_0),qubit_0)
qubit_111 = np.kron(np.kron(qubit_1,qubit_1),qubit_1)
c = random.random()
c




0.8866583978076755

In [11]:
qubit_final=(qubit_000 + c* qubit_111) * np.sqrt((1/(1+c**2)))
qubit_final

array([0.74823745, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.66343102])

In [2]:
import numpy as np
CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])
CNOT_tensor=np.reshape(CNOT_matrix, (2,2,2,2)) # reshaping it to make it in same basis as psi
H_matrix=(1/(np.sqrt(2)))*np.array([[1, 1],[1,-1]])




In [15]:

psi_1=np.zeros((2,2)) 
psi_2 = np.zeros((2,2))
psi_1[0,0] = psi_2[0,0] = 1 #psi_1 and psi_2 are 2d matrices with the 00 index as 1 (this gives me 00 qubit)

psi= np.kron(psi_1,psi_2) #psi is the tensor of in four basis like |i>|j>|k>|l>
psi=np.reshape(psi,(2,2,2,2)) #reshaping to 4 dimensional matrix to extract all indices in the basis ijkl

# using the fact that ψ′ijkl=∑i′Xii′ψi′jkl. where X is a 2d operator

psi=np.tensordot(CNOT_tensor,psi, ((2,3),(0,2))) #contraction 2nd index of cnot with 0th index of psi(0th qubit) and #contraction 3rd index of cnot with 2nd index of psi(2nd qubit)
psi=np.tensordot(CNOT_tensor,psi, ((2,3),(1,3))) #similarly #contraction 2nd index of cnot with 1st index of psi(1st qubit) and #contraction 3rd index of cnot with 3rd index of psi(3rd qubit)

psi = np.tensordot(H_matrix,psi,(1,0)) #contraction of 1st index of hadamard  with 0th index of psi
psi = np.tensordot(H_matrix,psi,(1,1)) #contraction of 1st index of hadamard  with 1st index of psi

projectors=[np.array([[1,0],[0,0]]),]

projected_2nd_qubit=np.tensordot(projectors,psi,(1,2)) # to measure the 2nd qubit , contracted 1st index of projectors to 2nd index of psi, just like hadamard
projected_3rd_qubit=np.tensordot(projectors,psi,(1,3)) # similar with the 3rd qubit



psi





array([[[[0.5, 0. ],
         [0. , 0. ]],

        [[0.5, 0. ],
         [0. , 0. ]]],


       [[[0.5, 0. ],
         [0. , 0. ]],

        [[0.5, 0. ],
         [0. , 0. ]]]])